<a href="https://colab.research.google.com/github/AngieB1988/Bigdata/blob/main/level2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [25]:
import os
# Find the latest version of spark 3.0  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.3.0'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 http://security.ubuntu.com/ubuntu bionic-security InRelease
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Hit:3 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:4 http://archive.ubuntu.com/ubuntu bionic InRelease
Ign:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:8 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:9 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:10 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:12 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Reading package lists... Done


In [26]:
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2022-09-19 15:07:23--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar.2’

postgresql-42.2.9.j 100%[===================>] 892.61K  4.92MB/s    in 0.2s    

2022-09-19 15:07:23 (4.92 MB/s) - ‘postgresql-42.2.9.jar.2’ saved [914037/914037]



In [27]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("CloudETL").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

In [28]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Wireless_v1_00.tsv.gz"
spark.sparkContext.addFile(url)

wireless_df = spark.read.csv(SparkFiles.get("amazon_reviews_us_Wireless_v1_00.tsv.gz"), sep="\t", header=True)
wireless_df.show(5)

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   16414143|R3W4P9UBGNGH1U|B00YL0EKWE|     852431543|LG G4 Case Hard T...|        Wireless|          2|            1|          3|   N|                Y|Looks good, funct...|2 issues  -  Once...| 2015-08-31|
|         US|   50800750|R15V54KBMTQWAY|B00XK95RPQ|     516894650|Selfie Stick Fibl...|        Wireless|          4|    

In [34]:
from pyspark import SparkFiles

vine_wireless_df = wireless_df.select(["review_id", "star_rating", "helpful_votes", "total_votes", "vine"])
vine_wireless_df.show(5)

+--------------+-----------+-------------+-----------+----+
|     review_id|star_rating|helpful_votes|total_votes|vine|
+--------------+-----------+-------------+-----------+----+
|R3W4P9UBGNGH1U|          2|            1|          3|   N|
|R15V54KBMTQWAY|          4|            0|          0|   N|
| RY8I449HNXSVF|          5|            0|          0|   N|
|R18TLJYCKJFLSR|          5|            0|          0|   N|
|R1NK26SWS53B8Q|          5|            0|          0|   N|
+--------------+-----------+-------------+-----------+----+
only showing top 5 rows



In [29]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Video_Games_v1_00.tsv.gz"
spark.sparkContext.addFile(url)

Video_Games_df = spark.read.csv(SparkFiles.get("amazon_reviews_us_Video_Games_v1_00.tsv.gz"), sep="\t", header=True)
Video_Games_df.show(5)


+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   12039526| RTIS3L2M1F5SM|B001CXYMFS|     737716809|Thrustmaster T-Fl...|     Video Games|          5|            0|          0|   N|                Y|an amazing joysti...|Used this for Eli...| 2015-08-31|
|         US|    9636577| R1ZV7R40OLHKD|B00M920ND6|     569686175|Tonsee 6 buttons ...|     Video Games|          5|    

In [30]:
from pyspark import SparkFiles

vine_Video_Games_df = Video_Games_df.select(["review_id", "star_rating", "helpful_votes", "total_votes", "vine"])
vine_Video_Games_df.show(5)

+--------------+-----------+-------------+-----------+----+
|     review_id|star_rating|helpful_votes|total_votes|vine|
+--------------+-----------+-------------+-----------+----+
| RTIS3L2M1F5SM|          5|            0|          0|   N|
| R1ZV7R40OLHKD|          5|            0|          0|   N|
|R3BH071QLH8QMC|          1|            0|          1|   N|
|R127K9NTSXA2YH|          3|            0|          0|   N|
|R32ZWUXDJPW27Q|          4|            0|          0|   N|
+--------------+-----------+-------------+-----------+----+
only showing top 5 rows



In [37]:

print(vine_wireless_df.count())
print(vine_Video_Games_df.count())
vine_table_Video_Games = vine_Video_Games_df.dropna()
vine_table_Video_Games= vine_table_Video_Games.na.drop()
vine_table_Video_Games= vine_table_Video_Games.dropDuplicates()
vine_table_wireless = vine_wireless_df.dropna()
vine_table_wireless = vine_table_wireless.na.drop()
vine_table_wirelss = vine_table_wireless.dropDuplicates()
print(vine_table_wireless.count())
print(vine_table_Video_Games.count())

9002021
1785997
9001881
1785997


In [38]:
#Video_Games
vine_table_Video_Games.describe().show()

+-------+--------------+------------------+------------------+------------------+-------+
|summary|     review_id|       star_rating|     helpful_votes|       total_votes|   vine|
+-------+--------------+------------------+------------------+------------------+-------+
|  count|       1785997|           1785997|           1785997|           1785997|1785997|
|   mean|          null| 4.059892597803915|2.2593492598251843| 3.757368573407458|   null|
| stddev|          null|1.3558117532896574|19.171285092306046|22.454391701509007|   null|
|    min|R10003I619LWL0|                 1|                 0|                 0|      N|
|    max| RZZZYCP48D6MH|                 5|               999|               999|      Y|
+-------+--------------+------------------+------------------+------------------+-------+



In [39]:
# Wireless
vine_table_wireless.describe().show()

+-------+--------------+------------------+------------------+------------------+-------+
|summary|     review_id|       star_rating|     helpful_votes|       total_votes|   vine|
+-------+--------------+------------------+------------------+------------------+-------+
|  count|       9001881|           9001881|           9001881|           9001881|9001881|
|   mean|          null|   3.8917818398177|0.8903038153914721|1.1573879947979762|   null|
| stddev|          null|1.4638208007143878|11.030083006409946|11.843696443744744|   null|
|    min|R10000B5EA0ZWP|                 1|                 0|                 0|      N|
|    max| RZZZY66N0DMNE|                 5|               993|               999|      Y|
+-------+--------------+------------------+------------------+------------------+-------+



In [40]:
# watches
from pyspark.sql.types import IntegerType
from pyspark import SparkFiles
vine_table_Video_Games = vine_table_Video_Games.withColumn("star_rating",Video_Games_df["star_rating"].cast(IntegerType()))
vine_table_Video_Games.groupBy("vine").avg("star_rating").show(truncate=False)

+----+-----------------+
|vine|avg(star_rating) |
+----+-----------------+
|Y   |4.074807737124213|
|N   |4.059856676690767|
+----+-----------------+



In [45]:
#Video_Games
from pyspark import SparkFiles
from pyspark.sql.functions import avg, col, desc
vine_table_Video_Games.groupBy("star_rating", "vine").count().sort(col("star_rating").desc()).show(truncate=False)

+-----------+----+-------+
|star_rating|vine|count  |
+-----------+----+-------+
|5          |Y   |1607   |
|5          |N   |1025317|
|4          |N   |316534 |
|4          |Y   |1712   |
|3          |Y   |718    |
|3          |N   |153146 |
|2          |N   |94603  |
|2          |Y   |194    |
|1          |N   |192106 |
|1          |Y   |60     |
+-----------+----+-------+



In [46]:
# wireless
from pyspark import SparkFiles
from pyspark.sql.functions import avg, col, desc
vine_table_wireless.groupBy("star_rating", "vine").count().sort(col("star_rating").desc()).show(truncate=False)

+-----------+----+-------+
|star_rating|vine|count  |
+-----------+----+-------+
|5          |N   |4818203|
|5          |Y   |6522   |
|4          |N   |1494784|
|4          |Y   |6511   |
|3          |N   |812301 |
|3          |Y   |2886   |
|2          |Y   |1094   |
|2          |N   |597223 |
|1          |Y   |468    |
|1          |N   |1261889|
+-----------+----+-------+



In [47]:
# wireless
from pyspark.sql.types import IntegerType
from pyspark import SparkFiles
vine_table_cameras = vine_table_wireless.withColumn("helpful_votes",wireless_df["helpful_votes"].cast(IntegerType()))
vine_table_cameras.groupBy("vine").avg("helpful_votes").show(truncate=False)

+----+------------------+
|vine|avg(helpful_votes)|
+----+------------------+
|Y   |4.398432583948287 |
|N   |0.8834780285828769|
+----+------------------+



In [48]:
#Video_Games
from pyspark.sql.types import IntegerType
from pyspark import SparkFiles
vine_table_Video_Games = vine_table_Video_Games.withColumn("helpful_votes",Video_Games_df["helpful_votes"].cast(IntegerType()))
vine_table_Video_Games.groupBy("vine").avg("helpful_votes").show(truncate=False)

+----+------------------+
|vine|avg(helpful_votes)|
+----+------------------+
|Y   |2.348403635516197 |
|N   |2.259134784302236 |
+----+------------------+

